In [35]:
import pandas as pd
import numpy as np
# from myUtils import pickle_load, symb_perf_stats_vectorized, symb_perf_stats_vectorized_v1
from myUtils import pickle_load, symb_perf_stats_vectorized_v2

file_close = 'df_close_clean'
path_dir = "C:/Users/ping/MyDrive/stocks/yfinance"
path_data_dump = path_dir + "/VSCode_dump/"

df_c = pickle_load(path_data_dump, file_close)
df_c = df_c[-252::]
format1 = 'df_c({}):\n{}\n{}'.format(len(df_c), df_c.head(3), df_c.tail(3))
# print(format1)

In [36]:
# %%timeit -n 20 -r 17
symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
    symb_perf_stats_vectorized_v2(df_c)

In [37]:
def perf_eval(df_close):
  '''
  df_close is a dataframe with date index, columns of symbols' closing price, and symbol as column name 
  '''

  symbols, period_yr, drawdown, UI, max_drawdown, returns_std, Std_UI, CAGR, CAGR_Std, CAGR_UI = \
      symb_perf_stats_vectorized_v2(df_close)

  caches_perf_stats_vect = []
  for symbol in symbols:
      # date_first = drawdown.index[0].strftime('%Y-%m-%d')
      # date_last = drawdown.index[-1].strftime('%Y-%m-%d')
      date_first = df_close.index[0].strftime('%Y-%m-%d')
      date_last = df_close.index[-1].strftime('%Y-%m-%d')

      cache = (symbol, date_first, date_last, period_yr, CAGR[symbol],
              UI[symbol], Std_UI[symbol], CAGR_Std[symbol], CAGR_UI[symbol])
      # append performance data (tuple) to caches_perf_stats (list)
      caches_perf_stats_vect.append(cache)
  column_names = ['symbol', 'first date', 'last date', 'Year', 'CAGR',
                  'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  # write symbols' performance stats to dataframe
  df_perf = pd.DataFrame(caches_perf_stats_vect, columns=column_names)

  _cols = ['CAGR', 'UI', 'Std/UI', 'CAGR/Std', 'CAGR/UI']
  grp_CAGRUI_mean = df_perf['CAGR/UI'].mean()
  grp_CAGRUI_std = df_perf['CAGR/UI'].std()
  grp_CAGRUI_mean_div_std = grp_CAGRUI_mean / grp_CAGRUI_std

  grp_CAGRStd_mean = df_perf['CAGR/Std'].mean()
  grp_CAGRStd_std = df_perf['CAGR/Std'].std()
  grp_CAGRStd_mean_div_std = grp_CAGRStd_mean / grp_CAGRStd_std

  grp_StdUI_mean = df_perf['Std/UI'].mean()
  grp_StdUI_std = df_perf['Std/UI'].std()
  grp_StdUI_mean_div_std = grp_StdUI_mean / grp_StdUI_std

  return df_perf, grp_StdUI_mean_div_std, grp_CAGRStd_mean_div_std, grp_CAGRUI_mean_div_std

In [48]:
df_perf, grp_StdUI_mean_div_std, grp_CAGRStd_mean_div_std, grp_CAGRUI_mean_div_std = perf_eval(df_c)
print(f'grp_StdUI_mean_div_std :{grp_StdUI_mean_div_std:.6f}')
print(f'grp_CAGRStd_mean_div_std :{grp_CAGRStd_mean_div_std:.6f}')
print(f'grp_CAGRUI_mean_div_std :{grp_CAGRUI_mean_div_std:.6f}')
df_perf.head()

grp_StdUI_mean_div_std :2.020599
grp_CAGRStd_mean_div_std :-0.257502
grp_CAGRUI_mean_div_std :0.083811


,symbol,first date,last date,Year,CAGR,UI,Std/UI,CAGR/Std,CAGR/UI
0,A,2021-12-20,2022-12-19,1.0,-0.016812,0.182402,0.122228,-0.754093,-0.092171
1,AA,2021-12-20,2022-12-19,1.0,-0.197264,0.401307,0.104174,-4.718584,-0.491553
2,AAL,2021-12-20,2022-12-19,1.0,-0.278613,0.251459,0.140915,-7.862803,-1.107984
3,AAON,2021-12-20,2022-12-19,1.0,-0.006359,0.266550,0.092308,-0.258456,-0.023858
4,AAP,2021-12-20,2022-12-19,1.0,-0.360861,0.215495,0.108891,-15.378359,-1.674567


In [59]:
my_symbols = ['AE', 'ABMD', 'RMBS', 'WNC']
my_cols = ['Std/UI',	'CAGR/Std', 'CAGR/UI']

In [ ]:
# # %%timeit -n 50 -r 20
# df_temp = df_c[_cols2]
# df_perf1, grp_StdUI_mean_div_std, grp_CAGRStd_mean_div_std, grp_CAGRUI_mean_div_std = perf_eval(df_temp)
# print(f'grp_Std/UI   mean/std: {grp_StdUI_mean_div_std:.6f}')
# print(f'grp_CAGR/Std mean/std: {grp_CAGRStd_mean_div_std:.6f}')
# print(f'grp_CAGR/UI  mean/std: {grp_CAGRUI_mean_div_std:.6f}')
# df_perf1

In [69]:
# %%timeit -n 50 -r 20
# select rows in my_symbols and columns in my_cols from df_perf
_df = df_perf.loc[df_perf['symbol'].isin(my_symbols)][my_cols]
grp_mean_div_grp_std = _df.mean() / _df.std()
print(f'{my_cols[0]}   grp_mean/grp_std: {grp_mean_div_grp_std[0]:.6f}')
print(f'{my_cols[1]} grp_mean/grp_std: {grp_mean_div_grp_std[1]:.6f}')
print(f'{my_cols[2]}  grp_mean/grp_std: {grp_mean_div_grp_std[2]:.6f}')
_df

Std/UI   grp_mean/grp_std: 4.378149
CAGR/Std grp_mean/grp_std: 1.976478
CAGR/UI  grp_mean/grp_std: 1.553355


,Std/UI,CAGR/Std,CAGR/UI
15,0.187439,4.862124,0.911351
39,0.184817,19.329833,3.572474
1761,0.127606,12.751041,1.627112
2243,0.121964,10.415199,1.270274


In [ ]:
df_SPY = df_c[['SPY']].copy()
df_SPY
df_perf, grp_StdUI_mean_std, grp_CAGRStd_mean_std, grp_CAGRUI_mean_std = perf_eval(df_SPY)
print(f'grp_StdUI_mean_std: {grp_StdUI_mean_std:.6f}')
print(f'grp_CAGRStd_mean_std: {grp_CAGRStd_mean_std:.6f}')
print(f'grp_CAGRUI_mean_std: {grp_CAGRUI_mean_std:.6f}')
df_perf.head()

In [ ]:
print(f'symbols:\n{symbols}')
print(f'period_yr:\n{period_yr}')
print(f'drawdown:\n{drawdown}')
print(f'UI:\n{UI}')
print(f'max_drawdown:\n{max_drawdown}')
print(f'returns_std:\n{returns_std}')
print(f'Std_UI:\n{Std_UI}')
print(f'CAGR:\n{CAGR}')
print(f'CAGR_Std:\n{CAGR_Std}')
print(f'CAGR_UI:\n{CAGR_UI}')